In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("test_image.png")

# Apply Yellow-White Mask
def yw_mask(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([20, 100, 100], dtype='uint8')
    upper_yellow = np.array([30, 255, 255], dtype='uint8')

    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(img_gray, 200, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_img = cv2.bitwise_and(img_gray, mask_yw)
    
    return mask_yw_img


# Add Gaussian Blur
def gauss_blur(mask_yw_img):
    return cv2.GaussianBlur(mask_yw_img, (5, 5), 0)


# Canny Edge Detection
def canny_edges(mask_yw_gauss):
    return cv2.Canny(mask_yw_gauss, 50, 150)


# Create and Apply ROI Mask
def roi_mask(canny_edges):
    width, height = canny_edges.shape[1], canny_edges.shape[0]

    roi_mask = np.zeros_like(canny_edges)
    roi_corners = [np.array([
        [width/9, height],
        [width/2 - width/8, height/2 + height/10],
        [width/2 + width/8, height/2 + height/10],
        [width - width/9, height]
    ], dtype=np.int32)]

    cv2.fillPoly(roi_mask, roi_corners, 255)
    
    roi_canny = cv2.bitwise_and(canny_edges, roi_mask)
    return roi_canny


def get_coordinates (image, params):
    slope, intercept = params 
    y1 = image.shape[0]     
    y2 = int(y1 * (3/5)) # Setting y2 at 3/5th from y1
    x1 = int((y1 - intercept) / slope) # Deriving from y = mx + c
    x2 = int((y2 - intercept) / slope) 
    
    return np.array([x1, y1, x2, y2])


def avg_lines(image, lines): 
    left = [] 
    right = [] 
    
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4)
          
        # Fit polynomial, find intercept and slope 
        params = np.polyfit((x1, x2), (y1, y2), 1)  
        slope = params[0] 
        y_intercept = params[1] 
        
        if slope < 0: 
            left.append((slope, y_intercept)) #Negative slope = left lane
        else: 
            right.append((slope, y_intercept)) #Positive slope = right lane
    
    # Avg over all values for a single slope and y-intercept value for each line
    
    left_avg = np.average(left, axis = 0) 
    right_avg = np.average(right, axis = 0) 
    
    # Find x1, y1, x2, y2 coordinates for left & right lines
    left_line = get_coordinates(image, left_avg) 
    right_line = get_coordinates(image, right_avg)
    
    return np.array([left_line, right_line])




# Hough Lines
def draw_hough_lines(img, roi_canny):
    line_img = np.zeros_like(img)
    lines = cv2.HoughLinesP(roi_canny,
                            2,
                            np.pi/180,
                            100,
                            minLineLength=40,
                            maxLineGap=25)

    averaged_lines = avg_lines(img, lines)
    for x1, y1, x2, y2 in averaged_lines:
        cv2.line(line_img, (x1, y1), (x2, y2), (0, 0, 255), 5)

    final_img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    return final_img


mask_yw_img = yw_mask(img)
mask_yw_gauss = gauss_blur(mask_yw_img)
canny_edges = canny_edges(mask_yw_gauss)
roi_canny = roi_mask(canny_edges)
final_img = draw_hough_lines(img, roi_canny)

cv2.imshow("Hough Lines", final_img)

cv2.waitKey(10000)
cv2.destroyAllWindows()
cv2.waitKey(1)